In [ ]:
import pandas
import numpy

# A)  Whole gene expression preparation

# B)  Dataframe preparation for CIBERSORT

In [ ]:
## need symbol for Cibersort
df_ciber = df_gexprProteinCoding_alNomination.copy()
print("shape:", df_ciber.shape)

#remove the column I don't want
df_ciber = df_ciber.drop(df_ciber.columns[[0, 2, 3]], axis=1)
print(df_ciber.shape)

#See how many NaN and remove them
print("NaN in symbol:", df_ciber['hgnc_symbol'].isna().sum())
df_ciber = df_ciber.dropna(axis=0)
print("after remove NaN:", df_ciber.shape)

#how many duplicate and remove
print("Duplicate in symbol:",df_ciber.duplicated(subset=["hgnc_symbol"]).sum())
df_ciber.drop_duplicates(subset = "hgnc_symbol", inplace = True)
print("after remove Duplicate:", df_ciber.shape)

#transpose the table 
df_ciber = numpy.transpose(df_ciber)

#Change head column & delete the name of the column index; choose symbol
df_ciber.columns = df_ciber.iloc[0]
df_ciber.columns.name = None
df_ciber = df_ciber.reset_index()

#delete row 0
df_ciber = df_ciber.drop(df_ciber.index[0])

#Column rename
df_ciber.rename(columns = {'index' : 'sample'}, inplace = True)
df_ciber.head(2)

In [ ]:
#Merge the two df to have isoform expression + whole gene expression 
df_cibersort = dfisof_3organs.merge(df_ciber, on= 'sample')

print(len(dfisof_3organs))
print(len(df_ciber))
print(len(df_cibersort))

#create a column to have the 2 information
df_cibersort['Info'] = df_cibersort[['Sample_Type', 'Primary_Site']].agg('-'.join, axis=1)

#move the column to rank 1 
col = df_cibersort.pop("Info")
df_cibersort.insert(1, "Info", col)

#Remove all columns between column index 1 to 18
df_cibersort.drop(df_cibersort.iloc[:, 2:25], inplace = True, axis = 1)

# cannot be log for Cibersort
col_list = df_cibersort.iloc[:,2:]
for col in col_list:
    df_cibersort[col] = pow(2,df_cibersort[col])+0.000001

df_cibersort.head(2)

## 1) All of them

In [ ]:
## For groupby, all of them
#choose the index column and transpose
df_cibersortgroup = df_cibersort.set_index('Info').T

#delete name of column index
df_cibersortgroup.columns.name = None

#delete the sample column
df_cibersortgroup = df_cibersortgroup.drop(df_cibersortgroup.index[0])
df_cibersortgroup.head()

In [ ]:
#grouby to have one column per condition to have small table
df_cibersortgroup = df_cibersortgroup.astype(float)
df_cibersortgroup = df_cibersortgroup.groupby(by=df_cibersortgroup.columns, axis=1).agg(numpy.median)
df_cibersortgroup.head()

In [ ]:
#be sure have not negative values
(df_cibersortgroup < 0).sum()

In [ ]:
#download table
df_cibersortgroup.to_csv('df_cibersort.txt', header=True, index=True, sep='\t')

## 1) High/Low expression

In [ ]:
#create a column to have the 2 information
HL_iso1['Inform'] = HL_iso1[['Sample_Type', 'Primary_Site', "L/H"]].agg('-'.join, axis=1)
HL_iso2['Inform'] = HL_iso2[['Sample_Type', 'Primary_Site', "L/H"]].agg('-'.join, axis=1)

#move the column to rank 1 
col = HL_iso1.pop("Inform")
HL_iso1.insert(0, "Inform", col)

col = HL_iso2.pop("Inform")
HL_iso2.insert(0, "Inform", col)
HL_iso2.head(2)

In [ ]:
#Merge the two df to have isoform expression + whole gene expression (inner to only keep what is similar)
df_cibersort_HL1 = pandas.merge(HL_iso1, df_cibersort, on= 'sample', how='inner')
df_cibersort_HL2 = pandas.merge(HL_iso2, df_cibersort, on= 'sample', how='inner')

print("HL_iso1:", HL_iso1.shape)
print("cybersort:", df_cibersort.shape)
print("final 1:",df_cibersort_HL1.shape)

print("HL_iso2:", HL_iso2.shape)
print("cybersort:", df_cibersort.shape)
print("final 2:", df_cibersort_HL2.shape)

#Remove all columns between column index 1 to 18
df_cibersort_HL1.drop(df_cibersort_HL1.iloc[:, 2:27], inplace = True, axis = 1)
df_cibersort_HL2.drop(df_cibersort_HL2.iloc[:, 2:27], inplace = True, axis = 1)

df_cibersort_HL1 = df_cibersort_HL1.drop(['sample'],  axis = 1) 
df_cibersort_HL2 = df_cibersort_HL2.drop(['sample'],  axis = 1) 
df_cibersort_HL1.head()

In [ ]:
## groupby, isoform expression 
#choose the index column and transpose
df_cibersort_HL1 = df_cibersort_HL1.set_index('Inform').T
df_cibersort_HL2 = df_cibersort_HL2.set_index('Inform').T

#delete name of column index
df_cibersort_HL1.columns.name = None
df_cibersort_HL2.columns.name = None

#delete the sample column
# df_cibersort_HL1 = df_cibersort_HL1.drop(df_cibersort_HL1.index[0])
# df_cibersort_HL2 = df_cibersort_HL2.drop(df_cibersort_HL2.index[0])

#grouby to have one column per condition to have small table
df_cibersort_HL1 = df_cibersort_HL1.astype(float)
df_cibersort_HL2 = df_cibersort_HL2.astype(float)

#groupby 
df_cibersort_HL1_group = df_cibersort_HL1.groupby(by=df_cibersort_HL1.columns, axis=1).agg(numpy.median)
df_cibersort_HL2_group = df_cibersort_HL2.groupby(by=df_cibersort_HL2.columns, axis=1).agg(numpy.median)
df_cibersort_HL1_group.head()

In [ ]:
#be sure have not negative value
print("ciber_hl1:",(df_cibersort_HL1_group < 0).sum().sum())
print("ciber_hl2:",(df_cibersort_HL2_group < 0).sum().sum())

#download table BUT TOO HEAVY FOR CIBERSORT
df_cibersort_HL1_group.to_csv('df_cibersort_HL1_group.txt', header=True, index=True, sep='\t')
df_cibersort_HL2_group.to_csv('df_cibersort_HL2_group.txt', header=True, index=True, sep='\t')

## for stat, separate per tissue and isoform

In [ ]:
df_cibersort_HL1

In [ ]:
# for stat, separate per tissue and isoform
panc_HL1_H = df_cibersort_HL1['Normal Tissue-Pancreas-High']
panc_HL1_L = df_cibersort_HL1['Normal Tissue-Pancreas-Low']
liver_HL1_H = df_cibersort_HL1['Normal Tissue-Liver-High']
liver_HL1_L = df_cibersort_HL1['Normal Tissue-Liver-Low']
kidney_HL1_H = df_cibersort_HL1['Normal Tissue-Kidney-High']
kidney_HL1_L = df_cibersort_HL1['Normal Tissue-Kidney-Low']


panc_HL2_H = df_cibersort_HL2['Normal Tissue-Pancreas-High']
panc_HL2_L = df_cibersort_HL2['Normal Tissue-Pancreas-Low']
liver_HL2_H = df_cibersort_HL2['Normal Tissue-Liver-High']
liver_HL2_L = df_cibersort_HL2['Normal Tissue-Liver-Low']
kidney_HL2_H = df_cibersort_HL2['Normal Tissue-Kidney-High']
kidney_HL2_L = df_cibersort_HL2['Normal Tissue-Kidney-Low']

paad_HL1_H = df_cibersort_HL1['Primary Tumor-Pancreas-High']
paad_HL1_L = df_cibersort_HL1['Primary Tumor-Pancreas-Low']
lihc_HL1_H = df_cibersort_HL1['Primary Tumor-Liver-High']
lihc_HL1_L = df_cibersort_HL1['Primary Tumor-Liver-Low']
kirc_HL1_H = df_cibersort_HL1['Primary Tumor-Kidney-High']
kirc_HL1_L = df_cibersort_HL1['Primary Tumor-Kidney-Low']

paad_HL2_H = df_cibersort_HL2['Primary Tumor-Pancreas-High']
paad_HL2_L = df_cibersort_HL2['Primary Tumor-Pancreas-Low']
lihc_HL2_H = df_cibersort_HL2['Primary Tumor-Liver-High']
lihc_HL2_L = df_cibersort_HL2['Primary Tumor-Liver-Low']
kirc_HL2_H = df_cibersort_HL2['Primary Tumor-Kidney-High']
kirc_HL2_L = df_cibersort_HL2['Primary Tumor-Kidney-Low']

paad_HL2_H.head()

In [ ]:
#download table
database = [panc_HL1_H, panc_HL1_L, 
            panc_HL2_H, panc_HL2_L, 
            liver_HL1_H, liver_HL1_L, 
            liver_HL2_H, liver_HL2_L,
            paad_HL1_H, paad_HL1_L, 
            paad_HL2_H, paad_HL2_L, 
            lihc_HL1_H, lihc_HL1_L, 
            lihc_HL2_H, lihc_HL2_L,
            kirc_HL1_H, kirc_HL1_L,
            kirc_HL2_H, kirc_HL2_L
            ]

database_labels = ['panc_HL1_H', 'panc_HL1_L', 
            'panc_HL2_H', 'panc_HL2_L', 
            'liver_HL1_H', 'liver_HL1_L', 
            'liver_HL2_H', 'liver_HL2_L',
            'paad_HL1_H', 'paad_HL1_L', 
            'paad_HL2_H', 'paad_HL2_L', 
            'lihc_HL1_H', 'lihc_HL1_L', 
            'lihc_HL2_H', 'lihc_HL2_L',
            'kirc_HL1_H', 'kirc_HL1_L',
            'kirc_HL2_H', 'kirc_HL2_L'
            ]
for i in range(len(database)): 
  w_df = database[i]
  w_label = database_labels[i]
  path = w_label + '.txt'

  w_df.to_csv(path, header=True, index=True, sep='\t')


In [ ]:
#download table
database = [df1, df2, df3, df4]

database_labels = ["name1", "name2", "name3", "name4"]
for i in range(len(database)): 
  w_df = database[i]
  w_label = database_labels[i]
  path = w_label + '.txt'

  w_df.to_csv(path, header=True, index=True, sep='\t')